## Cleaning Data

In [78]:
# Upload file to colab
from google.colab import files
uploads = files.upload()

In [79]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [80]:
# read file using pandas
df_q4 = pd.read_csv('tech_layoffs_til_Q4_2024.csv', sep=';', decimal=',')
df_q2 = pd.read_csv('tech_layoffs_Q2_2024.csv', sep=',')
df_q1 = pd.read_csv('tech_layoffs_til_Q1_2024.csv', sep=';')
df_sheet = pd.read_excel('tech_layoffs.xlsx')
df_loc = pd.read_csv('tech_layoffs_location.csv', sep=';')


In [81]:
cols_to_drop = ['Nr', 'Nr.', '#']

for df in [df_q4, df_q1, df_q2, df_sheet]:
    existing_cols = [c for c in cols_to_drop if c in df.columns]
    df.drop(columns=existing_cols, inplace=True)

rename_map = {'lat': 'latitude', 'lng': 'longitude'}

df_q1 = df_q1.rename(columns=rename_map)
df_q2 = df_q2.rename(columns=rename_map)
df_sheet = df_sheet.rename(columns=rename_map)

In [82]:
df_q4['Date_layoffs'] = pd.to_datetime(df_q4['Date_layoffs'], format='%d.%m.%y', errors='coerce')
df_q1['Date_layoffs'] = pd.to_datetime(df_q1['Date_layoffs'], errors='coerce')
df_q2['Date_layoffs'] = pd.to_datetime(df_q2['Date_layoffs'], errors='coerce')
df_sheet['Date_layoffs'] = pd.to_datetime(df_sheet['Date_layoffs'], errors='coerce')

In [83]:
df = pd.concat([df_q4, df_q1, df_q2, df_sheet], ignore_index=True)
df = df.drop_duplicates(subset=['Company', 'Date_layoffs', 'Laid_Off'])

print(f"Sukses! Total data: {len(df)} baris")
df.head()

Sukses! Total data: 2084 baris


,Company,Location_HQ,Region,USState,Country,Continent,Laid_Off,Date_layoffs,Percentage,Company_Size_before_Layoffs,Company_Size_after_layoffs,Industry,Stage,Money_Raised_in__mil,Year,latitude,longitude,Money_Raised_in_$_mil,State
0,Tamara Mellon,Los Angeles,NaN,California,USA,North America,20.0,2020-03-12,40.0,50.0,30.0,Retail,Series C,90,2020,34.053691,-118.242766,NaN,NaN
1,HopSkipDrive,Los Angeles,NaN,California,USA,North America,8.0,2020-03-13,10.0,80.0,72.0,Transportation,Unknown,45,2020,34.053691,-118.242766,NaN,NaN
2,Panda Squad,San Francisco,San Francisco Bay Area,California,USA,North America,6.0,2020-03-13,75.0,8.0,2.0,Consumer,Seed,1,2020,37.779259,-122.419329,NaN,NaN
3,Help.com,Austin,NaN,Texas,USA,North America,16.0,2020-03-16,100.0,16.0,NaN,Support,Seed,6,2020,30.271129,-97.743700,NaN,NaN
4,Inspirato,Denver,NaN,Colorado,USA,North America,130.0,2020-03-16,22.0,591.0,461.0,Travel,Series C,79,2020,39.739236,-104.984862,NaN,NaN


In [84]:
final_df = pd.merge(df, df_loc[['location_HQ', 'Country']],
                    left_on='Location_HQ',
                    right_on='location_HQ',
                    how='left',
                    suffixes=('', '_ref'))

final_df.head()

,Company,Location_HQ,Region,USState,Country,Continent,Laid_Off,Date_layoffs,Percentage,Company_Size_before_Layoffs,...,Industry,Stage,Money_Raised_in__mil,Year,latitude,longitude,Money_Raised_in_$_mil,State,location_HQ,Country_ref
0,Tamara Mellon,Los Angeles,NaN,California,USA,North America,20.0,2020-03-12,40.0,50.0,...,Retail,Series C,90,2020,34.053691,-118.242766,NaN,NaN,Los Angeles,USA
1,HopSkipDrive,Los Angeles,NaN,California,USA,North America,8.0,2020-03-13,10.0,80.0,...,Transportation,Unknown,45,2020,34.053691,-118.242766,NaN,NaN,Los Angeles,USA
2,Panda Squad,San Francisco,San Francisco Bay Area,California,USA,North America,6.0,2020-03-13,75.0,8.0,...,Consumer,Seed,1,2020,37.779259,-122.419329,NaN,NaN,San Francisco,USA
3,Help.com,Austin,NaN,Texas,USA,North America,16.0,2020-03-16,100.0,16.0,...,Support,Seed,6,2020,30.271129,-97.743700,NaN,NaN,Austin,USA
4,Inspirato,Denver,NaN,Colorado,USA,North America,130.0,2020-03-16,22.0,591.0,...,Travel,Series C,79,2020,39.739236,-104.984862,NaN,NaN,Denver,USA


In [85]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2084 entries, 0 to 2083
Data columns (total 21 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   Company                      2083 non-null   object        
 1   Location_HQ                  2084 non-null   object        
 2   Region                       538 non-null    object        
 3   USState                      1409 non-null   object        
 4   Country                      2084 non-null   object        
 5   Continent                    2084 non-null   object        
 6   Laid_Off                     1824 non-null   float64       
 7   Date_layoffs                 2084 non-null   datetime64[ns]
 8   Percentage                   1819 non-null   object        
 9   Company_Size_before_Layoffs  1670 non-null   object        
 10  Company_Size_after_layoffs   1603 non-null   object        
 11  Industry                     2084 non-null 

In [86]:
company_map = {
    'SaleSan Francisco Bay Areaorce': 'Salesforce',
    'SaleSan Franciscoorce': 'Salesforce',
    'TranSan Francisco Bay Areaix': 'Transfix',
    'TranSan Franciscoix': 'Transfix',
    'WeTranSan Francisco Bay Areaer': 'WeTransfer',
    'WeTranSan Franciscoer': 'WeTransfer',
    'MiSan Francisco Bay Areaits Market': 'Misfits Market',
    'MiSan Franciscoits Market': 'Misfits Market',

    '7shifts': '7Shifts',
    'Freshbooks': 'FreshBooks',
    'BitTitan': 'BitTitan',
    'ByteDance': 'Bytedance',

    'Branch Metrics': 'Branch',
    'Branch.io': 'Branch',
    'CureFit': 'Cure.fit',
    'Riot Gaming': 'Riot Games',
    'Stash Financial': 'Stash',
    'Lightspeed Company': 'Lightspeed Commerce',
    'Take-Two': 'Take-Two Interactive',
    'GoTo Group': 'GoTo',
    'Booking.com': 'Booking Holdings',
    'Buy.com / Rakuten': 'Rakuten',

    'NanoString Tech…': 'NanoString Technologies',
    'Integral Ad Scien…': 'Integral Ad Science',
    'Community Gami…': 'Community Gaming',
    'Repertoire Immu…': 'Repertoire Immunity',
    'Trove Recommer…': 'Trove Recommerce',
    'Unstoppable Do.': 'Unstoppable Domains',
    'Eden / Managed …': 'Eden',
    'nan': 'Unknown'
}

final_df['Company'] = final_df['Company'].replace(company_map)

final_df['Company'] = final_df['Company'].str.strip()

print("Cek Salesforce:", final_df[final_df['Company'].str.contains('Salesforce', case=False, na=False)]['Company'].unique())
print("Cek Transfix:", final_df[final_df['Company'].str.contains('Transfix', case=False, na=False)]['Company'].unique())

print("\n--- TOP 10 COMPANIES (CLEANED) ---")
print(final_df['Company'].value_counts().head(10))

Cek Salesforce: ['Salesforce']
Cek Transfix: ['Transfix']

--- TOP 10 COMPANIES (CLEANED) ---
Company
Google        10
Amazon         8
Salesforce     8
Microsoft      6
Gopuff         6
Swiggy         6
Rivian         6
Meta           5
Lyft           5
New Relic      5
Name: count, dtype: int64


In [87]:
industry_map = {
    'Transportion': 'Transportation',
    'e-commerce': 'E-commerce',
    'cloud': 'Cloud'
}

final_df['Industry'] = final_df['Industry'].replace(industry_map)

In [88]:
final_df['Location_HQ'] = final_df['Location_HQ'].str.strip()

location_map = {
    'Bejing': 'Beijing',
    'Ferdericton': 'Fredericton',
    'Forster City': 'Foster City',

    'Tampa Bay': 'Tampa',
    'Frankfurt am Main': 'Frankfurt',
    'Washington DC': 'Washington D.C.'
}

final_df['Location_HQ'] = final_df['Location_HQ'].replace(location_map)

In [89]:
continent_map = {
    'Oceana': 'Oceania'
}

final_df['Continent'] = final_df['Continent'].replace(continent_map)

In [90]:
final_df['Money_Raised_in__mil'] = final_df['Money_Raised_in__mil'].fillna(final_df['Money_Raised_in_$_mil'])

cols_to_drop = ['Money_Raised_in_$_mil', 'location_HQ', 'State', 'Country_ref']
final_df = final_df.drop(columns=cols_to_drop)

In [91]:
def clean_currency(x):
  if isinstance(x, str):
    x = x.replace('$', '').replace('USD', '').replace(',', '.').replace('%', '')
  return x

target_cols = ['Percentage', 'Company_Size_before_Layoffs', 'Company_Size_after_layoffs', 'Money_Raised_in__mil']

for col in target_cols:
  final_df[col] = final_df[col].apply(clean_currency)
  final_df[col] = pd.to_numeric(final_df[col], errors='coerce')

final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2084 entries, 0 to 2083
Data columns (total 17 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   Company                      2082 non-null   object        
 1   Location_HQ                  2084 non-null   object        
 2   Region                       538 non-null    object        
 3   USState                      1409 non-null   object        
 4   Country                      2084 non-null   object        
 5   Continent                    2084 non-null   object        
 6   Laid_Off                     1824 non-null   float64       
 7   Date_layoffs                 2084 non-null   datetime64[ns]
 8   Percentage                   1819 non-null   float64       
 9   Company_Size_before_Layoffs  1670 non-null   float64       
 10  Company_Size_after_layoffs   1603 non-null   float64       
 11  Industry                     2084 non-null 

In [92]:
# Handling Missing Value on Region & US State
final_df = final_df.drop(columns=['Region'], errors='igonre')

final_df.loc[final_df['Country'] != 'USA', 'USState'] = 'Non-US'
final_df['USState'] = final_df['USState'].fillna('Unknown')

In [93]:
# Handling Missing Value on Laid Off, Percentage, Company Size
# Putting null value by columns relation
# Formula: Laid_Off = Size * (Percentage /100)

mask_layoff = final_df['Laid_Off'].isna() & final_df['Company_Size_before_Layoffs'].notna() & final_df['Percentage'].notna()
final_df.loc[mask_layoff, 'Laid_Off'] = final_df.loc[mask_layoff, 'Company_Size_before_Layoffs'] * (final_df.loc[mask_layoff, 'Percentage'] / 100)

mask_size = final_df['Company_Size_before_Layoffs'].isna() & final_df['Laid_Off'].notna() & final_df['Percentage'].notna()
final_df.loc[mask_size, 'Company_Size_before_Layoffs'] = final_df.loc[mask_size, 'Laid_Off'] / (final_df.loc[mask_size, 'Percentage'] /100)

final_df['Laid_Off'] = final_df['Laid_Off'].round(0)
final_df['Company_Size_before_Layoffs'] = final_df['Company_Size_before_Layoffs'].round(0)

final_df['Money_Raised_in__mil'] = final_df['Money_Raised_in__mil'].fillna(0)

print('Info after Advanced Cleaning:')
final_df.info()

Info after Advanced Cleaning:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2084 entries, 0 to 2083
Data columns (total 16 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   Company                      2082 non-null   object        
 1   Location_HQ                  2084 non-null   object        
 2   USState                      2084 non-null   object        
 3   Country                      2084 non-null   object        
 4   Continent                    2084 non-null   object        
 5   Laid_Off                     1824 non-null   float64       
 6   Date_layoffs                 2084 non-null   datetime64[ns]
 7   Percentage                   1819 non-null   float64       
 8   Company_Size_before_Layoffs  1670 non-null   float64       
 9   Company_Size_after_layoffs   1603 non-null   float64       
 10  Industry                     2084 non-null   object        
 11  Stage        

In [94]:
final_df = final_df.dropna(subset=['Company'])

final_df['Month_Name'] = final_df['Date_layoffs'].dt.strftime('%B')
final_df['Quarter'] = final_df['Date_layoffs'].dt.to_period('Q').astype(str)

print("Final Data Shape:", final_df.shape)
final_df.head()

Final Data Shape: (2082, 18)


,Company,Location_HQ,USState,Country,Continent,Laid_Off,Date_layoffs,Percentage,Company_Size_before_Layoffs,Company_Size_after_layoffs,Industry,Stage,Money_Raised_in__mil,Year,latitude,longitude,Month_Name,Quarter
0,Tamara Mellon,Los Angeles,California,USA,North America,20.0,2020-03-12,40.0,50.0,30.0,Retail,Series C,90.0,2020,34.053691,-118.242766,March,2020Q1
1,HopSkipDrive,Los Angeles,California,USA,North America,8.0,2020-03-13,10.0,80.0,72.0,Transportation,Unknown,45.0,2020,34.053691,-118.242766,March,2020Q1
2,Panda Squad,San Francisco,California,USA,North America,6.0,2020-03-13,75.0,8.0,2.0,Consumer,Seed,1.0,2020,37.779259,-122.419329,March,2020Q1
3,Help.com,Austin,Texas,USA,North America,16.0,2020-03-16,100.0,16.0,NaN,Support,Seed,6.0,2020,30.271129,-97.743700,March,2020Q1
4,Inspirato,Denver,Colorado,USA,North America,130.0,2020-03-16,22.0,591.0,461.0,Travel,Series C,79.0,2020,39.739236,-104.984862,March,2020Q1


In [95]:
final_df = final_df.dropna(subset=['Company'])
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2082 entries, 0 to 2082
Data columns (total 18 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   Company                      2082 non-null   object        
 1   Location_HQ                  2082 non-null   object        
 2   USState                      2082 non-null   object        
 3   Country                      2082 non-null   object        
 4   Continent                    2082 non-null   object        
 5   Laid_Off                     1824 non-null   float64       
 6   Date_layoffs                 2082 non-null   datetime64[ns]
 7   Percentage                   1819 non-null   float64       
 8   Company_Size_before_Layoffs  1670 non-null   float64       
 9   Company_Size_after_layoffs   1603 non-null   float64       
 10  Industry                     2082 non-null   object        
 11  Stage                        2082 non-null   obj

In [96]:
final_df.to_csv('cleaned_tech_layoffsv1.csv', index=False)

In [97]:
df_analyst = final_df.copy()